In [ ]:
import intake
import iris
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf

from holoviews.operation.datashader import regrid

gv.extension('bokeh')

Guidence on wind shear for aviation
===================================
We're expecting high shear conditions over the UK this week. This analysis will identify risk regions and issue shear warning accoringly.

### Calcualte shear in the Mogreps G runs
I'm doing to pull in data from the Met Office global ensemble forcast, Mogreps G

In [ ]:
intake.gui

I'm going to calculate shear on pressure levels for Mogreps G.

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
# direction = intake.cat.mo_aws_earth.mogreps_g.wind_from_direction_at_pressure.read()
# speed = intake.cat.mo_aws_earth.mogreps_g.wind_speed_at_pressure.read()
direction = iris.load_cube('wind_direction.nc')
speed = iris.load_cube('wind_speed.nc')

In [ ]:
from opscentretools import sheartools
shear = sheartools.calculate_shear(speed, direction)
shear

In [ ]:
# from jade_utils.iris_tools import estimate_cube_size
# datasize = estimate_cube_size(shear)
# datasize

### Exploratory plot of windshear

We need to investigate the windshear field to get a feel for it

First we'll create a plottable object from the data field.

In [ ]:
width = 1000
height = 600
this_cube = shear[0]
shear_gv = gv.Dataset(this_cube, [coord.name() for coord in this_cube.dim_coords])
shear_plot = regrid(shear_gv.to(gv.Image, ['longitude', 'latitude'], dynamic=True), width=width, height=height)
shear_plot = shear_plot.opts(colorbar=True, width=width, height=height, cmap='Viridis', projection=ccrs.PlateCarree())

Next we get the weather warning tool

In [ ]:
from opscentretools import annotable
orange_warning_pen, orange_warning = annotable.warning("orange")

and some coastlines

In [ ]:
coastlines = gv.feature.coastline

And plot them all together

In [ ]:
shear_interactive = shear_plot * coastlines * orange_warning
shear_interactive

### Consult with the chief
I'd like to highlight areas in my advice, but I'd like to check with the cheif to see if they're on the same page.

In [ ]:
annotable.make(shear_interactive, websocket_origin='pangeo-dev.informaticslab.co.uk')

In [ ]:
orange_warning_pen.element.data

### Check the average wind shear profile

In [ ]:
max_wind_shear = shear.collapsed(["forecast_period", "forecast_reference_time", "realization", "latitude", "longitude"], iris.analysis.MAX)
max_wind_shear

### Calculate proportion of MOGREPS-G run that exceeds 15 m/s and 25 m/s shear limits for aviation

In [ ]:
from opscentretools import sheartools
prob_shear_15 = sheartools.calculate_ensemble_exceedence(shear, threshold=15)
prob_shear_25 = sheartools.calculate_ensemble_exceedence(shear, threshold=25)

In [ ]:
def interactive_image(cube):
    dataset = gv.Dataset(cube, [coord.name() for coord in cube.dim_coords])
    image = regrid(dataset.to(gv.Image, ['longitude', 'latitude'], dynamic=True))
    coastlines = gv.feature.coastline
    return image.opts(colorbar=True, cmap='magma', projection=ccrs.PlateCarree(), responsive=True, min_height=600, aspect=2, tools=['hover'])

In [ ]:
def interactive_image(cube, cmap='viridis', coastlines=True, coastline_color='grey', projection=ccrs.PlateCarree, tools=['hover'], min_height=600, **opts):
    dataset = gv.Dataset(cube, [coord.name() for coord in cube.dim_coords])
    image = regrid(dataset.to(gv.Image, ['longitude', 'latitude'], dynamic=True))
    
    options = {
        'cmap': cmap,        
        'responsive': True,
        'projection': projection(),
        'colorbar': True,
        'min_height': min_height,
        'aspect': 2,
        'tools': tools
    }
    
    if coastlines:
        return image.opts(**options, **opts) * gv.feature.coastline.opts(line_color=coastline_color)
    else:
        return image.opts(**options, **opts)

In [ ]:
from opscentretools import plotting

In [ ]:
# fifteen_interactive = interactive_image(prob_shear_15, cmap='greens')
# fifteen_interactive * gv.feature.coastline
fifteen_interactive = plotting.interactive_plot(prob_shear_15, cmap='reds')
fifteen_interactive

In [ ]:
twentyfive_interactive = interactive_image(prob_shear_25)
twentyfive_interactive